<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import requests as r
import json

In [2]:
data = {"series":[{"key":"(4.2.3) Before the vote count, did the Returning Officer compare the ballot paper account received from the Presiding Officers with the number of ballot papers in ballot boxes, the number of unused and spoilt ballot papers, and the tendered votes list?","label":"(4.2.3) Before the vote count, did the Returning Officer compare the ballot paper account received from the Presiding Officers with the number of ballot papers in ballot boxes, the number of unused and spoilt ballot papers, and the tendered votes list?","data":[{"value":2},{"value":2},{"value":30}]}],"common":{"data":[{"key":"Not Sure","label":"Not Sure"},{"key":"No","label":"No"},{"key":"Yes","label":"Yes"}],"metadata":{"type":"text"}}}

In [3]:
output = r.get('http://localhost:4000/draft/results-piechart.json').json()

In [4]:
#output['legend']

In [5]:
getLegend = lambda x : [y['key'] for y in x]
getData = lambda x : [y['value'] for y in x]

In [6]:
#output['legend']['data'] = getLegend(data['common']['data'])
legends = getLegend(data['common']['data'])

In [7]:
yesno = ['Yes', 'No']
if set(yesno).issubset(legends):
    if set(['Not Sure']).issubset(legends):
        yesno.append('Not Sure')
    legend_df = pd.Series(legends).to_frame('legend').reset_index()
    yesno_df = pd.Series(yesno).to_frame('order').reset_index()
    yesno_df = yesno_df.merge(legend_df, left_on='order', right_on='legend', suffixes=['_a', '_b'], how='outer')
    ordered_legends = list(yesno_df.sort_values(by='index_a')['index_b'])
    ordered_series = []
    ordered_keys = []
    for i in ordered_legends:
        ordered_series.append(data['series'][0]['data'][i])
        ordered_keys.append(data['common']['data'][i])
    data['series'][0]['data'] = ordered_series
    data['common']['data'] = ordered_keys
print(data['common']['data'])

[{'key': 'Yes', 'label': 'Yes'}, {'key': 'No', 'label': 'No'}, {'key': 'Not Sure', 'label': 'Not Sure'}]


In [8]:
output['title']['text'] = str(sum(getData(data['series'][0]['data'])))
#output['title']['subtext'] = "Add Subtext Here"

In [9]:
output['series'][0]['data'] = []
for i,d in enumerate(data['series'][0]['data']):
    output['series'][0]['data'].append({'value':d['value'],'name':data['common']['data'][i]['key']})

In [10]:
getName = lambda x : [y['name'] for y in x]
output['legend']['data'] = getName(output['series'][0]['data'])

In [11]:
json = json.dumps(output)
# f = open("results-piechart.json","w")
f = open("../charts-generator/new-json/testing.json","w")
f.write(json)
f.close()